In [16]:
import pandas as pd
import numpy as np

from util import comparar_columnas, eliminar_caracteristicas, prueba_umbral
from feature_selection import *

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
import joblib
import json
import os

In [31]:
final = pd.read_csv('final_dataframe.csv', index_col=False)
selected_features = ['PP',
'n_5-15cm',
'slope',
'Bulkd.5-15cm',
'n_100-200cm',
'Tex_Class.100-200cm',
'theta_s_0-5cm',
'theta_s_15-30cm',
'valor_humedad_suelo1',
'Valor']
final = final[selected_features]

KeyError: "['n_5-15cm', 'Bulkd.5-15cm', 'n_100-200cm', 'Tex_Class.100-200cm', 'theta_s_0-5cm', 'theta_s_15-30cm'] not in index"

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score
import joblib
import os
import json

def entrenar_y_guardar_modelos(df, columna_objetivo, ruta_guardado):
    # Separar características y variable objetivo
    X = df.drop(columns=[columna_objetivo])
    y = df[columna_objetivo]
    
    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    # Definir los modelos y sus hiperparámetros para GridSearchCV
    modelos = {
        'RandomForest': {
            'modelo': RandomForestClassifier(random_state=42),
            'parametros': {
                'classifier__n_estimators': [100, 200, 300, 500],
                'classifier__max_depth': [3, 5, 7, 10, 15, 20, None],
                'classifier__min_samples_split': [2, 5, 10, 15, 20],
                'classifier__min_samples_leaf': [1, 2, 4, 6, 8],
                'classifier__max_features': ['sqrt', 'log2', None],
                'classifier__bootstrap': [True, False],
                'classifier__random_state': [42],
                'classifier__max_leaf_nodes': [None, 50, 100, 200],
                'classifier__min_weight_fraction_leaf': [0.0, 0.1, 0.2],
                'classifier__min_impurity_decrease': [0.0, 0.01, 0.05],
                'classifier__n_jobs': [-1]
            }
        },
        'SVC': {
            'modelo': SVC(probability=True, random_state=42),
            'parametros': {
                'classifier__C': [0.1, 1, 2, 10, 100],
                'classifier__kernel': ['rbf'],
                'classifier__gamma': ['scale', 'auto', 0.01, 0.1, 1, 10, 20, 30, 50, 100],
                'classifier__random_state': [42]
            }
        },
        'XGBoost': {
            'modelo': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
            'parametros': {
                # Parámetros de estructura del árbol
                'classifier__max_depth': [3, 5, 7, 9, 11],
                'classifier__min_child_weight': [1, 3, 5, 7],
                'classifier__gamma': [0, 0.1, 0.2, 0.3, 0.4],
                
                # Parámetros de regulación
                'classifier__learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
                'classifier__n_estimators': [100, 200, 300, 500],
                
                # Parámetros de regularización
                'classifier__reg_alpha': [0, 0.1, 0.5, 1.0],
                'classifier__reg_lambda': [0.1, 0.5, 1.0, 2.0],
                
                
                # Otros parámetros
                'classifier__random_state': [42]
            }
        }
    }
    
    metricas = {}
    
    for nombre, item in modelos.items():
        modelo = item['modelo']
        parametros = item['parametros']
        
        # Crear un Pipeline que incluye el escalado y el modelo
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', modelo)
        ])
        
        # Realizar la búsqueda de hiperparámetros
        grid = GridSearchCV(
            pipeline,
            param_grid=parametros,
            cv=3,
            scoring='accuracy',
            n_jobs=-1,
            verbose=1
        )
        grid.fit(X_train, y_train)
        mejor_modelo = grid.best_estimator_
        
        # Guardar el mejor modelo en un archivo .pkl
        nombre_archivo_modelo = os.path.join(ruta_guardado, f"{nombre}_mejor_modelo.pkl")
        joblib.dump(mejor_modelo, nombre_archivo_modelo)
        
        # Predecir en el conjunto de prueba
        y_pred = mejor_modelo.predict(X_test)
        
        # Calcular métricas
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sensibilidad = tp / (tp + fn)  # Recall
        especificidad = tn / (tn + fp)  # Specificity
        precision = precision_score(y_test, y_pred)
        exactitud = accuracy_score(y_test, y_pred)
        
        metricas[nombre] = {
            'Sensibilidad': sensibilidad,
            'Especificidad': especificidad,
            'Precisión': precision,
            'Exactitud': exactitud,
            'Mejores Hiperparámetros': grid.best_params_
        }
        
        # Añadir los nombres de las características al modelo
        mejor_modelo.named_steps['classifier'].feature_names = X_train.columns.tolist()
    
    # Guardar las métricas en un archivo .txt
    nombre_archivo_metricas = os.path.join(ruta_guardado, 'metricas_modelos.txt')
    with open(nombre_archivo_metricas, 'w') as file:
        json.dump(metricas, file, indent=4)
    
    return metricas

In [33]:
target = 'Valor'
path_guardar = 'resultados_ga\ga_xgboost_mutflip_0_5_mutprob_0_3_cpx_0_8'
entrenar_y_guardar_modelos(final, target, path_guardar)

Fitting 3 folds for each of 151200 candidates, totalling 453600 fits
Fitting 3 folds for each of 50 candidates, totalling 150 fits
Fitting 3 folds for each of 32000 candidates, totalling 96000 fits


c:\Users\ivo\.conda\envs\geotiff\lib\site-packages\xgboost\core.py:158: UserWarning: [07:06:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'RandomForest': {'Sensibilidad': 0.8559498956158664,
  'Especificidad': 0.8068669527896996,
  'Precisión': 0.82,
  'Exactitud': 0.8317460317460318,
  'Mejores Hiperparámetros': {'classifier__bootstrap': True,
   'classifier__max_depth': 15,
   'classifier__max_features': None,
   'classifier__max_leaf_nodes': 200,
   'classifier__min_impurity_decrease': 0.0,
   'classifier__min_samples_leaf': 1,
   'classifier__min_samples_split': 5,
   'classifier__min_weight_fraction_leaf': 0.0,
   'classifier__n_estimators': 500,
   'classifier__n_jobs': -1,
   'classifier__random_state': 42}},
 'SVC': {'Sensibilidad': 0.7933194154488518,
  'Especificidad': 0.7532188841201717,
  'Precisión': 0.7676767676767676,
  'Exactitud': 0.7735449735449735,
  'Mejores Hiperparámetros': {'classifier__C': 2,
   'classifier__gamma': 0.1,
   'classifier__kernel': 'rbf',
   'classifier__random_state': 42}},
 'XGBoost': {'Sensibilidad': 0.8830897703549061,
  'Especificidad': 0.8347639484978541,
  'Precisión': 0.846,

In [34]:
from pubagging import PUBagging

data = final
data['Valor'] = final.Valor
data_aux_1 = data[data.Valor == 1]
data_aux_0 = data[data.Valor == 0]
data_aux_0 = data_aux_0.drop(columns='Valor')
data_aux_1 = data_aux_1.drop(columns='Valor')

landslide_samples = data_aux_1# Example landslide samples as DataFrame
unlabeled_samples = data_aux_0# Example unlabeled samples as DataFrame

pu_bagging = PUBagging(num_iterations=5, sample_ratio=0.4, random_state=42)

pu_bagging.fit(landslide_samples, unlabeled_samples)

probabilities = pu_bagging.predict_proba(unlabeled_samples)
print("Predicted probabilities:", probabilities)

threshold = 0.5 

# Filtrar muestras no deslizamiento por debajo del umbral
non_landslide_indices = np.where(probabilities < threshold)[0]
selected_non_landslide_samples = unlabeled_samples.iloc[non_landslide_indices]

# Ejemplo de cómo podrías utilizar las muestras seleccionadas
print("Muestras no deslizamiento seleccionadas:")
print(selected_non_landslide_samples)

# Se crean los nuevos dataframe para el reentreno donde tenemos data y data_y como input y output respectivamente
selected_non_landslide_samples['Valor'] = 0
data_aux_1['Valor'] = 1

data = pd.concat([data_aux_1, selected_non_landslide_samples])
data_y = data['Valor']
#data = data.drop(columns=['Valor'])

Predicted probabilities: [0.2 0.  0.2 ... 0.4 0.2 0.8]
Muestras no deslizamiento seleccionadas:
      PIRange_Bulkd.0-5cm.tif  PIRange_Bulkd.100-200cm.tif  \
0                       0.609                        0.689   
1                       0.609                        0.689   
2                       0.595                        0.753   
3                       0.589                        0.682   
4                       0.659                        0.689   
...                       ...                          ...   
1820                    0.588                        0.687   
1821                    0.594                        0.684   
1822                    0.625                        0.690   
1823                    0.593                        0.691   
1824                    0.592                        0.702   

      PIRange_Bulkd.15-30cm.tif  PIRange_Bulkd.30-60cm.tif  \
0                         0.483                      0.641   
1                         0.483    

C:\Users\ivo\AppData\Local\Temp\ipykernel_14580\4043113540.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_non_landslide_samples['Valor'] = 0


In [ ]:
path_guardar = 'resultados_ga\ga_xgboost_mutflip_0_5_mutprob_0_3_cpx_0_8\pubagging'
entrenar_y_guardar_modelos(data, target, path_guardar)

Fitting 3 folds for each of 151200 candidates, totalling 453600 fits


In [30]:
import joblib
import numpy as np
from sklearn.metrics import accuracy_score, recall_score
from sklearn.utils import resample

def evaluar_modelo(ruta_modelo, X_test, y_test, n_bootstrap=1000, alpha=0.95):
    """
    Evalúa un modelo guardado en un archivo .pkl y calcula su exactitud y recall con intervalos de confianza.

    :param ruta_modelo: Ruta del archivo .pkl del modelo
    :param X_test: Características de prueba
    :param y_test: Etiquetas de prueba
    :param n_bootstrap: Número de muestras bootstrap para calcular el intervalo de confianza
    :param alpha: Nivel de confianza para el intervalo (por defecto 0.95)
    :return: Diccionario con exactitud, recall y sus intervalos de confianza
    """
    # Cargar el modelo
    modelo = joblib.load(ruta_modelo)
    
    # Predecir en el conjunto de prueba
    y_pred = modelo.predict(X_test)
    
    # Calcular métricas
    exactitud = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    # Bootstrap para intervalos de confianza
    exactitud_bootstrap = []
    recall_bootstrap = []
    
    for _ in range(n_bootstrap):
        X_resampled, y_resampled = resample(X_test, y_test)
        y_pred_resampled = modelo.predict(X_resampled)
        exactitud_bootstrap.append(accuracy_score(y_resampled, y_pred_resampled))
        recall_bootstrap.append(recall_score(y_resampled, y_pred_resampled))
    
    # Calcular percentiles para el intervalo de confianza
    lower_bound = (1.0 - alpha) / 2.0
    upper_bound = 1.0 - lower_bound
    
    exactitud_intervalo = (np.percentile(exactitud_bootstrap, lower_bound * 100), 
                           np.percentile(exactitud_bootstrap, upper_bound * 100))
    recall_intervalo = (np.percentile(recall_bootstrap, lower_bound * 100), 
                        np.percentile(recall_bootstrap, upper_bound * 100))
    exactitud_error = (exactitud_intervalo[1] - exactitud_intervalo[0]) / 2
    recall_error = (recall_intervalo[1] - recall_intervalo[0]) / 2
    resultados = {
        'Exactitud': exactitud,
        'Exactitud Intervalo': exactitud_intervalo,
        'Recall': recall,
        'Recall Intervalo': recall_intervalo
    }
    print(f"Exactitud: {exactitud:.4f} +- {exactitud_error:.4f}")
    print(f"Recall: {recall:.4f} +- {recall_error:.4f}")
    return resultados

# Ejemplo de uso
ruta_modelo = 'resultados_ga\ga_randomforest_mutflip_0_3_mutprob_0_3_cpx_0_6\pubagging\RandomForest_mejor_modelo.pkl'
X = final.drop(columns=[target])
y = final[target]
    
    # Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
resultados = evaluar_modelo(ruta_modelo, X_test, y_test)
print(resultados)

Exactitud: 0.9397 +- 0.0159
Recall: 0.9770 +- 0.0128
{'Exactitud': 0.9396825396825397, 'Exactitud Intervalo': (0.9238095238095239, 0.9555555555555556), 'Recall': 0.9770354906054279, 'Recall Intervalo': (0.9622621690185508, 0.9879281714382682)}
